<!-- Trabalho Desenvolvido no Curso da Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Fundamentos de Linguagem Python - Do Básico a Aplicações de IA</font>
# <font color='blue'>Mini-Projeto 6</font>
# <font color='blue'>Modelo de Classificação Para Análise de Sentimentos</font>

## 1. Definição do Problema de Negócio

É a fundação do projeto. Nesta etapa, traduzimos uma necessidade da empresa em um objetivo claro para a Ciência de Dados. Definimos o que queremos resolver (automatizar a classificação de reviews de produtos, por exemplo), por que é importante (reduzir custos, agilizar a tomada de decisão) e como o sucesso será medido.

**Definição:**

Uma empresa de e-commerce deseja automatizar a análise de feedback de seus clientes. Atualmente, a análise é feita manualmente, o que é um processo lento, caro e que não escala com o volume de reviews recebidos diariamente.

**Objetivo:** 

Criar um modelo de Machine Learning que classifique automaticamente os reviews de produtos como **'positivo'** ou **'negativo'**.

**Benefícios Esperados:**
* **Eficiência:** Reduzir o tempo e o custo da análise de feedback.
* **Tomada de Decisão Rápida:** Permitir que as equipes de produto e marketing identifiquem rapidamente produtos com problemas ou oportunidades de melhoria.
* **Priorização:** Direcionar reviews negativos para a equipe de suporte ao cliente de forma prioritária, melhorando a experiência do consumidor.

## 2. Importação dos Pacotes

Esta é a fase de preparação do ambiente de trabalho. Aqui, carregamos todas as ferramentas e bibliotecas Python necessárias (como Pandas para manipulação de dados, Scikit-learn para modelagem e Matplotlib/Seaborn para visualizações) que serão utilizadas ao longo do projeto.

In [3]:
# Instala o pacote watermark

%pip install -q -U watermark

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Manipulação de dados e visualização
import re
import pandas as pd
import numpy as np
import unicodedata
import seaborn as sns
import matplotlib.pyplot as plt

# Pré-Processamento e Machine Learning
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib

In [ ]:
# Configurações de visualização
sns.set_style('whitegrid')
%matplotlib inline

In [ ]:
%reload_ext watermark
%watermark -a "Data Science Academy"

In [ ]:
%watermark --iversions

## 3. Carregando e Compreendendo os Dados

É o primeiro contato com a matéria-prima do projeto. Lemos os dados de uma fonte (no nosso caso, um arquivo CSV) para um DataFrame do Pandas e realizamos uma verificação inicial para entender sua estrutura.

In [ ]:
# Define o nome do arquivo
nome_arquivo_csv = 'dataset.csv'

In [ ]:
# Carrega o dataset
df_dsa = pd.read_csv(nome_arquivo_csv)

In [ ]:
# Shape
df_dsa.shape

In [ ]:
# Primeiras linhas
df_dsa.head()

In [ ]:
# Amostra dos dados
df_dsa.sample(10)

In [ ]:
# Últimas linhas
df_dsa.tail()

## 4. Análise Exploratória de Dados (EDA)

A fase de investigação. Aqui, "mergulhamos" nos dados para descobrir padrões, anomalias e insights. Utilizamos gráficos (como a contagem de sentimentos positivos vs. negativos) e estatísticas para entender a distribuição e as características dos dados antes de qualquer modificação.

In [ ]:
# Info
df_dsa.info()

In [ ]:
print("\nVerificando valores ausentes:\n")
print(df_dsa.isnull().sum())

In [ ]:
print("\nDistribuição dos Sentimentos:\n")
sns.countplot(x = 'sentimento', data = df_dsa)
plt.title('Distribuição das Classes de Sentimento')
plt.show()

## 5. Limpeza de Dados

Nesta etapa corrigimos os problemas identificados na EDA. Isso inclui tratar valores ausentes (por exemplo, removendo as linhas correspondentes), garantir que os dados estejam no formato correto e aplicar a função de limpeza de texto (dsa_limpa_texto) para normalizar os reviews, removendo acentos, pontuações e caracteres indesejados.

In [ ]:
# Remover linhas com valores ausentes
print(f"\nTamanho original do DataFrame: {len(df_dsa)}")
df_dsa.dropna(subset = ['texto_review'], inplace = True)
print(f"Tamanho do DataFrame após remover nulos: {len(df_dsa)}")

In [ ]:
df_dsa.shape

In [ ]:
df_dsa.head()

In [ ]:
# Função de limpeza de texto 
def dsa_limpa_texto(texto):
    
    """
    Função completa de limpeza de texto:
    1. Converte para minúsculas.
    2. Remove acentos e cedilha.
    3. Remove pontuações, números e caracteres especiais.
    4. Remove espaços extras.
    """
    
    # Garante que o texto não seja nulo (caso haja algum NaN no DataFrame)
    if not isinstance(texto, str):
        return ""

    # --- PASSO 1: Normalizar e remover acentos ---
    # Normaliza para a forma 'NFKD' que separa o caractere da acentuação
    # e depois remove os acentos (Nonspacing Mark)
    texto_sem_acentos = ''.join(c for c in unicodedata.normalize('NFKD', texto) if unicodedata.category(c) != 'Mn')

    # --- PASSO 2: Limpeza com Regex ---
    # Converter para minúsculas
    texto_limpo = texto_sem_acentos.lower()
    
    # Manter apenas letras e espaços. A remoção de acentos já foi feita.
    texto_limpo = re.sub(r'[^a-z\s]', '', texto_limpo)
    
    # Remover espaços extras
    texto_limpo = re.sub(r'\s+', ' ', texto_limpo).strip()
    
    return texto_limpo

In [ ]:
# Aplica a função de limpeza
df_dsa['texto_limpo'] = df_dsa['texto_review'].apply(dsa_limpa_texto)

In [ ]:
df_dsa.head()

https://docs.python.org/3/library/re.html

## 6. Engenharia de Atributos

É a etapa onde transformamos dados brutos em "features" (atributos) úteis para o modelo. No nosso projeto, isso envolveu a criação da coluna texto_limpo e, mais importante, a conversão das classes de texto ('positivo', 'negativo') em um formato numérico (sentimento_label: 1, 0), que é o que o algoritmo de Machine Learning consegue entender.

In [ ]:
df_dsa.head()

In [ ]:
# Mapear o sentimento para valores numéricos (Engenharia de Atributos)
df_dsa['sentimento_label'] = df_dsa['sentimento'].map({'positivo': 1, 'negativo': 0})

In [ ]:
print("\nDataFrame após a limpeza e mapeamento:\n")
df_dsa[['texto_limpo', 'sentimento_label']].head()

## 7. Divisão em Dados de Treino de Teste

Um passo essencial para treinamento e avaliação do modelo. Separamos o conjunto de dados em duas partes: uma maior (treino), que o modelo usará para aprender, e uma menor (teste), que será mantida "escondida" do modelo para simular dados novos e avaliar sua real performance após o treinamento.

In [ ]:
# Definir variáveis X (entrada) e y (saída)
X = df_dsa['texto_limpo']
y = df_dsa['sentimento_label']

In [ ]:
# Dividir os dados em treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size = 0.25, random_state = 42, stratify = y)

## 8. Pipeline de Modelagem Preditiva

Aqui, construímos uma esteira de produção automatizada para o nosso modelo. O Pipeline do Scikit-learn encapsula todas as etapas de pré-processamento (vetorização TF-IDF, padronização com StandardScaler) e o modelo final (Regressão Logística), garantindo que os mesmos passos sejam aplicados de forma consistente nos dados de treino e nos novos dados.

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [ ]:
# Pipeline 
pipeline = Pipeline([
    
    ('tfidf', TfidfVectorizer(stop_words = ['de', 'a', 'o', 'que', 'e', 'do', 'da', 'em', 'um'])),
    
    ('scaler', StandardScaler(with_mean = False)),
    
    ('logreg', LogisticRegression(solver = 'liblinear', random_state = 42, max_iter = 1000)) 
])

In [ ]:
type(pipeline)

O pipeline é composto por três etapas sequenciais, cada uma com um nome ('tfidf', 'scaler', 'logreg') e uma função específica.

### 8.1. Vetorização dos Dados de Texto ('tfidf', TfidfVectorizer(...))

O que faz? Esta é a primeira etapa, chamada 'tfidf'. Ela usa o TfidfVectorizer para converter o texto bruto em números. Ele transforma cada review em um vetor numérico, onde cada número representa a importância de uma palavra naquele texto em relação a todos os outros textos. Palavras comuns como 'de', 'a', 'o', etc., são ignoradas (stop_words).

### 8.2. Padronização de Dados ('scaler', StandardScaler(with_mean=False))

O que faz? A segunda etapa, 'scaler', pega os vetores numéricos criados pela etapa anterior e os padroniza. O StandardScaler ajusta a escala de todos os números para que tenham uma variância semelhante. O parâmetro with_mean=False é essencial aqui, pois a matriz de dados do TF-IDF é "esparsa" (cheia de zeros), e essa opção preserva essa característica, evitando problemas de memória e computação.

### 8.3. Modelo de Machine Learning ('logreg', LogisticRegression(...))

O que faz? A etapa final, 'logreg', é o modelo de classificação em si. A LogisticRegression recebe os dados numéricos, já padronizados, e aprende a fazer a previsão final: classificar o texto como "positivo" ou "negativo".


O grande benefício do pipeline é a automação e a consistência. Ao treinar este pipeline, ele aprende a fazer todas as três coisas. Depois, quando você apresenta um novo texto para ele prever, o pipeline automaticamente executa a mesma sequência exata de vetorização, padronização e classificação, garantindo que não haja erros no pré-processamento.

### 8.4. Otimização de Hiperparâmetros

É o ajuste fino do modelo. Usando GridSearchCV, testamos sistematicamente várias combinações de configurações (hiperparâmetros) para o pipeline, a fim de encontrar a combinação que resulta na melhor performance possível.

In [ ]:
# Definir o grid de hiperparâmetros para otimização
parametros_grid = {
    'tfidf__max_features': [500, 1000, 2000],
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'logreg__C': [0.1, 1, 10],
    'logreg__penalty': ['l1', 'l2'],
    'logreg__max_iter': [5000, 6000]
}

In [ ]:
# Configurar o GridSearchCV
grid_search = GridSearchCV(
    pipeline,              # Pipeline com as etapas de pré-processamento e modelo
    parametros_grid,       # Dicionário com as combinações de hiperparâmetros a serem testadas
    cv = 5,                # Número de divisões para validação cruzada (5-fold cross-validation)
    n_jobs = -1,           # Usa todos os núcleos disponíveis do processador para acelerar o processo
    scoring = 'accuracy',  # Métrica usada para avaliar o desempenho de cada combinação (aqui, acurácia)
    verbose = 1            # Nível de detalhamento do output durante a execução (1 exibe progresso básico)
)

Validação cruzada é uma técnica usada para avaliar o desempenho de um modelo dividindo o conjunto de dados em várias partes (ou “folds”). O modelo é treinado em algumas dessas partes e testado em outras, de forma rotativa. Isso permite medir o desempenho de forma mais confiável e geral, evitando que o resultado dependa apenas de uma única divisão dos dados.

### 8.5. Treinamento do Modelo

Nesta etapa, alimentamos o pipeline com os dados de treino. O GridSearchCV executa o processo de .fit(), onde o algoritmo aprende os padrões que conectam o texto dos reviews aos seus respectivos sentimentos.

In [ ]:
print("\nIniciando o treinamento do modelo com otimização de hiperparâmetros...\n")
grid_search.fit(X_treino, y_treino)

In [ ]:
print("\nMelhores hiperparâmetros encontrados:\n")
print(grid_search.best_params_)

In [ ]:
# Obter o melhor modelo
melhor_modelo_dsa = grid_search.best_estimator_

In [ ]:
type(melhor_modelo_dsa)

## 9. Avaliação do Modelo e Interpretação de Métricas

É a "prova final". Usamos o conjunto de teste (os dados que o modelo nunca viu) para fazer previsões e compará-las com os resultados reais. Métricas como Acurácia, Relatório de Classificação e a Matriz de Confusão nos dizem quão bem o modelo está generalizando e se ele atende aos objetivos de negócio.

In [ ]:
# Previsões no conjunto de teste
y_pred = melhor_modelo_dsa.predict(X_teste)

In [ ]:
# Calcular as métricas de avaliação
acuracia = accuracy_score(y_teste, y_pred)
report = classification_report(y_teste, y_pred, target_names = ['Negativo', 'Positivo'])

In [ ]:
print(f"\nAcurácia do Modelo: {acuracia:.2%}\n")
print("Relatório de Classificação:\n")
print(report)

In [ ]:
# Visualizar a Matriz de Confusão
cm = confusion_matrix(y_teste, y_pred)
sns.heatmap(cm, annot = True, fmt = 'd', cmap = 'Blues',
            xticklabels = ['Negativo', 'Positivo'],
            yticklabels = ['Negativo', 'Positivo'])
plt.xlabel('Previsão')
plt.ylabel('Verdadeiro')
plt.title('Matriz de Confusão')
plt.show()

**1. Acurácia (Accuracy)**

De todos os reviews (positivos e negativos) que o modelo analisou, qual a porcentagem total de acertos?

Em nosso contexto: Se você passou 100 reviews pelo modelo e ele acertou a classificação de 92 deles (independentemente de serem positivos ou negativos), a acurácia é de 92%. É a medida mais geral de performance.

**2. Precisão (Precision)**

Esta métrica é calculada para cada classe.

Precisão para a classe 'Positivo': Dos reviews que o modelo rotulou como 'Positivos', quantos eram realmente Positivos?

Em nosso contexto: Se o modelo marcou 10 reviews como "Positivos", mas apenas 9 deles eram de fato positivos (e 1 era um erro, um falso positivo), a precisão para a classe 'Positivo' é de 90%.

Importância: Uma alta precisão aqui significa que você pode confiar na etiqueta "Positivo". Se você for usar esses reviews em um material de marketing, por exemplo, não quer mostrar acidentalmente um review negativo.

Precisão para a classe 'Negativo': Dos reviews que o modelo rotulou como 'Negativos', quantos eram realmente Negativos?

Em nosso contexto: Se o modelo marcou 10 reviews como "Negativos" e todos os 10 eram de fato negativos, a precisão para a classe 'Negativo' é de 100%.

Importância: Essencial para a equipe de suporte. Uma alta precisão aqui garante que o tempo da equipe não seja gasto analisando reviews que na verdade eram positivos (falsos negativos para o time de suporte).

**3. Recall**

Esta métrica também é calculada para cada classe.

Recall para a classe 'Positivo': De todos os reviews que realmente eram Positivos, quantos o seu modelo conseguiu encontrar?

Em nosso contexto: Se existiam 12 reviews positivos no total, mas o seu modelo só conseguiu identificar 9 deles (deixando 3 passarem despercebidos), o recall para a classe 'Positivo' é de 75% (9 de 12).

Importância: Um alto recall aqui garante que você está capturando a maior parte do feedback positivo. Você não está perdendo a oportunidade de identificar clientes satisfeitos.

Recall para a classe 'Negativo': De todos os reviews que realmente eram Negativos, quantos o seu modelo conseguiu encontrar?

Em nosso contexto: Se existiam 10 clientes insatisfeitos (reviews negativos), e o modelo identificou 9 deles, o recall para 'Negativo' é de 90%. Ele "deixou escapar" um cliente insatisfeito.

Importância: Talvez a métrica mais crítica para a gestão de crises. Um alto recall para 'Negativo' é vital para garantir que quase nenhuma reclamação passe despercebida e que a equipe de suporte possa agir.

**4. F1-Score**

Qual é o balanço equilibrado entre a Precisão e o Recall para cada classe?

Em nosso contexto: É uma única nota que combina as duas métricas anteriores. O F1-Score para a classe 'Positivo' só será alto se tanto a precisão quanto o recall para 'Positivo' forem altos. Ele evita o cenário onde um modelo é ótimo em uma métrica, mas péssimo em outra. É frequentemente a melhor métrica para avaliar a performance do modelo em uma classe específica.

**5. Support (Suporte)**

Quantos reviews de cada classe realmente existiam no meu conjunto de teste?

Em nosso contexto: Se o relatório mostra um support de 58 para 'Positivo' e 62 para 'Negativo', significa que no seu conjunto de dados de teste havia 58 reviews positivos e 62 negativos.

Importância: Dá contexto aos resultados. Se o support de uma classe for muito baixo, as métricas de performance para essa classe são menos confiáveis, pois foram calculadas com base em poucos exemplos.

In [ ]:
# Se estivermos satisfeitos com a performance do modelo, salvamos em disco
joblib.dump(melhor_modelo_dsa, 'modelo_sentimento_dsa_v1.joblib')

In [ ]:
# Pode deletar o modelo treinado e removê-lo da memória
del melhor_modelo_dsa

**Nota: Normalmente aqui termina o trabalho de um Cientista de Dados. A etapa a seguir é, normalmente, responsabilidade de um Engenheiro de Machine Learning.**

## 10. MLOps, Deploy e Uso do Modelo

A etapa final, onde o modelo começa a gerar valor real. "Deploy" significa colocar o modelo treinado em produção. No nosso caso, simulamos isso criando a função dsa_prever_sentimento, que encapsula o pipeline treinado e pode ser usada para classificar novos reviews de forma rápida e automática.

Leia sobre MLOps no Capítulo 11 do curso.

In [ ]:
# Carregar o modelo a partir do disco
modelo_dsa_deploy = joblib.load('modelo_sentimento_dsa_v1.joblib')

In [ ]:
type(modelo_dsa_deploy)

In [ ]:
# Criar novos dados para simular o uso em produção
novos_reviews = [
    "A bateria do celular não dura nada, péssima compra.",
    "Chegou antes do prazo e o produto é de ótima qualidade! Estou muito feliz.",
    "O serviço de atendimento foi rápido e eficiente.",
    "Não recomendo, veio faltando peças e a cor estava errada."
]

In [ ]:
# Função para prever o sentimento de novos reviews 
def dsa_prever_sentimento(reviews):
    
    """
    Recebe uma lista de textos de review e retorna a previsão de sentimento.
    O objeto 'melhor_modelo_dsa' (pipeline) cuida de todos os passos internos.
    """
    
    # 1. 'reviews' entra no pipeline
    # 2. TF-IDF é aplicado internamente
    # 3. StandardScaler é aplicado internamente
    # 4. LogisticRegression faz a previsão
    previsoes = modelo_dsa_deploy.predict(reviews)
    
    # Mapeia o resultado numérico de volta para texto
    sentimentos = ['Negativo' if p == 0 else 'Positivo' for p in previsoes]
    
    # Exibe os resultados
    for review, sentimento in zip(reviews, sentimentos):
        print(f"\nReview: '{review}'\nSentimento Previsto: {sentimento}\n---")

In [ ]:
# Executar a função de deploy com os novos dados
print("\n--- Iniciando Classificação de Novos Reviews (Deploy com Pipeline Completo) ---\n")
dsa_prever_sentimento(novos_reviews)

**Projeto concluído, projeto entregue, cliente feliz, partimos para o próximo projeto! :-)**

# Fim